In [ ]:
import tkinter
import tkintermapview


In [1]:
from data_collect_GUI import GUI

gui = GUI()
gui.run()

In [2]:
import tkinter as tk
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import serial
import numpy as np
import time
import pandas as pd

class GUI:
    def __init__(self, s_port='com20', baud=9600, max_val = 100):
        self.start_time = time.time()
        self.ser = serial.Serial()
        ser_open = False
        attempts = 0

        #attempt to connect
        while not ser_open and attempts < 5:
            try:
                self.ser = serial.Serial(s_port, baud, timeout=None)
                ser_open = True
            except:
                print("Failed to open serial port: ", attempts)
                
            if not ser_open:
                # self.ser.close()
                # print("Closed Serial Port")
                time.sleep(0.1)
                attempts += 1
        
        # verify the serial port is open
        if not ser_open:
            print("Failed to open serial port - closing")
            return
        
        # serial port is open - create the main application window
        self.root = tk.Tk()
        self.root.title("Real-time Serial Plotter")

        self.top_frame = tk.Frame(self.root)
        self.top_frame.pack()
        self.mid_frame = tk.Frame(self.root)
        self.mid_frame.pack(side=tk.TOP)
        self.btm_frame = tk.Frame(self.root)
        self.btm_frame.pack(side=tk.BOTTOM)

        self.t = 0.0
        self.dt = 0.01
        self.speed_input_pwm = 1500
        self.steer_input_pwm = 1500
        self.speed_input_pwms = []
        self.steer_input_pwms = []

        self.max_vals = max_val

        self.pwm_df = pd.DataFrame(columns=['time', 'status', 'steer', 'throttle', 'control'])
        self.pwm_times = []
        self.throttles = []
        self.steers = []
        self.statuses = []
        self.controls = []
        
        self.df = pd.DataFrame(columns=['time', 'lat', 'lon', 'alt', 'speed', 'heading', 'fusion_calibration'])
        self.gps_times = []
        self.lats = []
        self.lons = []
        self.alts = []
        self.speeds = []
        self.headings = []

        # create a list of figure and axes objects to plot the data
        self.figures = []
        self.axes = []
        self.n_figures = 4
        for _ in range(self.n_figures):
            fig = Figure(figsize=(4, 2), dpi=100)
            ax = fig.add_subplot(1, 1, 1)
            ax.grid()
            self.figures.append(fig)
            self.axes.append(ax)

        # create a list of canvas objects for the plots and add them to the window
        self.canvases = []
        for fi, fig in enumerate(self.figures):
            if fi % 2 == 0:
                canvas = FigureCanvasTkAgg(fig, master=self.top_frame)
            else:
                canvas = FigureCanvasTkAgg(fig, master=self.mid_frame)
            canvas.get_tk_widget().pack(side=tk.LEFT, fill=tk.BOTH)
            self.canvases.append(canvas)


        self.entry_canvas = tk.Canvas(self.btm_frame, width=600, height=300)

        self.speed_label = tk.Label(self.btm_frame, text="Enter Speed Command (900-2100)")
        self.entry_canvas.create_window(150,60,window=self.speed_label)

        self.speed_entry = tk.Entry(self.root)
        self.entry_canvas.create_window(300,60, window=self.speed_entry)

        label_widget2 = tk.Label(self.btm_frame, text="Enter Steer Command (900-2100)")
        self.entry_canvas.create_window(150,100,window=label_widget2)

        self.steer_entry = tk.Entry(self.btm_frame)
        self.entry_canvas.create_window(300,100, window=self.steer_entry)

        self.submit_button = tk.Button(text='Submit', command=self.submit_data)
        self.entry_canvas.create_window(225,150, window=self.submit_button)
        
        self.reset_button = tk.Button(text='Reset', command=self.reset_data)
        self.entry_canvas.create_window(225,200, window=self.reset_button)
        
        self.oval = self.entry_canvas.create_oval(50, 50, 0, 100)  # Create a circle on the Canvas

        self.entry_canvas.pack(side=tk.LEFT, fill=tk.BOTH)

        
        self.median_text = tk.StringVar()
        self.median_text.set("Median Throttle Command: " + str(np.median(self.throttles)) + "\nMedian Steer Command: " + str(np.median(self.speeds)))
        self.label = tk.Label(self.root, textvariable=self.median_text)
        self.label.pack()
    
    def submit_data(self):
        try:
            speed_pwm_in = int(self.speed_entry.get())
            steer_pwm_in = int(self.steer_entry.get())
            
            if speed_pwm_in >= 900 and speed_pwm_in <= 2100:
                self.speed_input_pwm = speed_pwm_in
                
            if steer_pwm_in >= 900 and steer_pwm_in <= 2100:
                self.steer_input_pwm = steer_pwm_in
        
        except:
            tk.messagebox.showinfo('Error', "Your data was not formatted properly")  
        
        self.speed_entry.delete(0, tk.END)
        self.steer_entry.delete(0, tk.END)

    def reset_data(self):
        self.steer_input_pwm = 1500
        self.speed_input_pwm = 1500

    # create a function to read the serial data and update the plots
    def update_plots(self):
        try:
            # capture time
            c_time = time.time() - self.start_time
            # read the serial data
            while self.ser.in_waiting:
                bytes = self.ser.readline()
                # print("bytes: ", bytes)
                try:
                    decoded_strings = str(bytes[0:len(bytes)-2].decode("utf-8")).split(',')
                    # print("decoded_strings: ", decoded_strings)
                    if decoded_strings[0] == 'G':
                        decoded_strings[0] = c_time
                        # print("GPS: ", decoded_strings)
                        self.gps_ingest_line(decoded_strings)
                    elif decoded_strings[0] == 'P':
                        decoded_strings[0] = c_time
                        # print("PWM: ", decoded_strings)
                        self.pwm_ingest_line(decoded_strings)
                    elif decoded_strings[0] == 'E':
                        print("ERR: ", decoded_strings)
                    else:
                        print("Error: BAD STRING: ",  decoded_strings)
                except:
                    print("GUI didn't log Serial RF message correctly: ", bytes)
            
            
            if self.controls[-1] > 1700:
                self.entry_canvas.itemconfig(self.oval, fill="green")
            else:
                self.entry_canvas.itemconfig(self.oval, fill="red")
                
            self.axes[0].clear()
            self.axes[0].plot(self.pwm_times, self.throttles, label="Received")
            # self.axes[0].plot(self.pwm_times, self.controls, label="Control")
            self.axes[0].plot(self.pwm_times, self.speed_input_pwms, label="Set")
            self.axes[0].grid()
            self.axes[0].set_ylabel("Speed PWM")
            self.axes[0].set_title("Speed Commands vs Time")
            self.axes[0].legend(loc='upper left')
            self.axes[0].set_ylim([900, 2100])
            
            
            self.axes[1].clear()
            self.axes[1].plot(self.gps_times, self.speeds, label="Speed")
            self.axes[1].grid()
            self.axes[1].set_ylabel("Speed (m/s)")
            self.axes[1].set_title("Vehicle Speed vs Time")
            # axes[1].legend()
            self.axes[1].set_ylim([-5, 15])
            
            
            self.axes[2].clear()
            self.axes[2].plot(self.pwm_times, self.steers, label="Received")
            # self.axes[2].plot(self.pwm_times, self.controls, label="Control")
            self.axes[2].plot(self.pwm_times, self.steer_input_pwms, label="Set")
            self.axes[2].grid()
            self.axes[2].set_ylabel("Steering PWM")
            self.axes[2].set_title("Steering Commands vs Time")
            self.axes[2].legend(loc='upper left')
            self.axes[2].set_ylim([900, 2100])
            
            
            self.axes[3].clear()
            self.axes[3].plot(self.gps_times, self.headings, label='Heading')
            self.axes[3].grid()
            self.axes[3].set_ylabel("Heading (Degs)")
            self.axes[3].set_title("Heading Rate vs Time")
            # axes[3].legend()
            self.axes[3].set_ylim([0.0, 360.0])
            
            self.median_text.set("Median Throttle Command: " + str(np.median(self.throttles)) + "\nMedian Steer Command: " + str(np.median(self.speeds)))
            
            # update the plots on the canvases
            for canvas in self.canvases:
                canvas.draw()
        except:
            pass
        
        # schedule the function to run again after 100ms
        self.root.after(int(self.dt*1000), self.update_plots)
        
    def pwm_ingest_line(self, line):
            # [0.01695728302001953, '1', '-1', '1502', '1497', '$']
            # print("pwm line: ", line)
            # print("time: ", line[0])
            line[1] = float(line[1])
            # print("status: ", line[1])
            line[2] = float(line[2]) / 10**3
            # print("Time: ", line[2])
            line[3] = float(line[3])
            # print("control: ", line[3])
            line[4] = float(line[4])
            # print("steer: ", line[4])
            line[5] = float(line[5])
            # print("throttle: ", line[5])            
            
            # print("line: ", line)
            # self.pwm_df.loc[len(self.pwm_df)] = line[:-1]
            
            # append for plotting
            self.statuses.append(line[1])
            self.pwm_times.append(line[2])
            self.controls.append(line[3])
            self.steers.append(line[4])
            self.throttles.append(line[5])
            # track gui inputs here too
            self.speed_input_pwms.append(self.speed_input_pwm)
            self.steer_input_pwms.append(self.steer_input_pwm)
            

            while len(self.pwm_times) > self.max_vals:
                self.pwm_times.pop(0)
            while len(self.statuses) > self.max_vals:
                self.statuses.pop(0)
            while len(self.controls) > self.max_vals:
                self.controls.pop(0)
            while len(self.steers) > self.max_vals:
                self.steers.pop(0)
            while len(self.throttles) > self.max_vals:
                self.throttles.pop(0)

            while len(self.speed_input_pwms) > self.max_vals:
                self.speed_input_pwms.pop(0)
            while len(self.steer_input_pwms) > self.max_vals:
                self.steer_input_pwms.pop(0)
        
    def gps_ingest_line(self, line):
            # print("gps line: ", line)
            # ['G', '394265191', '-1198031066', '1466181', '18', '0', '0', '$']
            line[1] = float(line[1]) / 10**3
            line[2] = float(line[2]) / 10**7
            # print("lat: ", line[1])
            line[3] = float(line[3]) / 10**7
            # print("lon: ", line[2])
            line[4] = float(line[4]) / 10**3
            # print("alt: ", line[3])
            line[5] = float(line[5]) / 10**3
            # print("speed: ", line[4])
            line[6] = float(line[6]) / 10**5
            # print("heading: ", line[5])
            # print("cal status: ", line[6])
            
            # self.gps_df.loc[len(self.gps_df)] = line[:-1]
            
            self.gps_times.append(line[1])
            self.lats.append(line[2])
            self.lons.append(line[3])
            self.alts.append(line[4])
            self.speeds.append(line[5])
            self.headings.append(line[6])
            
            while len(self.gps_times) > self.max_vals:
                self.gps_times.pop(0)
            while len(self.lats) > self.max_vals:
                self.lats.pop(0)
            while len(self.lons) > self.max_vals:
                self.lons.pop(0)
            while len(self.alts) > self.max_vals:
                self.alts.pop(0)
            while len(self.speeds) > self.max_vals:
                self.speeds.pop(0)
            while len(self.headings) > self.max_vals:
                self.headings.pop(0)

    def run(self):
        # start the function to update the plots
        self.update_plots()

        # start the main event loop
        self.root.mainloop()
        
    def __del__(self):
        try:
            self.ser.close()
            print("Closed Serial Port")
        except:
            print("Failed to close serial port")

if __name__ == "__main__":

    gui = GUI(max_val = 100)
    gui.run()

c:\Users\sgtas\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\sgtas\AppData\Local\Programs\Python\Python39\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


GUI didn't log Serial RF message correctly:  b'P,1,520\xffP,1,1697,1505,14\xf6P,1,1653,1503,1458,1536,$\r\n'
GUI didn't log Serial RF message correctly:  b'P,1,75760,352,1558,$\r\n'
GUI didn't log Serial RF message correctly:  b'G,897850270,1379559,3133,9884390,0,$\r\n'
GUI didn't log Serial RF message correctly:  b'G,817850199,1379597,3191,9865208,0,$\r\n'
GUI didn't log Serial RF message correctly:  b'G,82280,394390468,-1159,1548,$\r\n'
GUI didn't log Serial RF message correctly:  b'P,1,8\x13b\x8a\xaa\xaa\x9ab"j\n'
GUI didn't log Serial RF message correctly:  b'P,1,88878\r\n'
GUI didn't log Serial RF message correctly:  b'G,89258,394391107,-119785116583316,0,$\r\n'
Error: BAD STRING:  ['P$']
GUI didn't log Serial RF message correctly:  b'P,1,92623,1502577,$\r\n'
GUI didn't log Serial RF message correctly:  b'G,92679,394390697,-1197850938804,0,$\r\n'
GUI didn't log Serial RF message correctly:  b'P,1,93184,1505,1445$\r\n'
GUI didn't log Serial RF message correctly:  b'P,1,93272,1505,1